# Operator class

## Abstract

We present a Python class whose instances model simplicial operators. It supports their composition and action on computable simplices. The `Operator` class forms the basis of the `simplicial_operators` package.

In [ ]:
pip install simplicial_operators

## Simplicial sets

A **[simplicial set](https://en.wikipedia.org/wiki/Simplicial_set)** $X$ is a collection of sets $\{X_n\}_{n \geq 0}$ together with **degeneracy maps** and **face maps**

\begin{equation*}
s_i : X_n \to X_{n+1} \qquad d_i : X_n \to X_{n-1}
\end{equation*}
for $i = 0, \dots, n$ satisfying the **simplicial identities**:

\begin{align*}
d_i d_j &= d_{j-1} d_i  &\text{ if } i &< j.  \\
d_i s_j &= s_{j-1}d_i    &\text{ if } i &< j. \\
d_i s_j &= \text{id}  &\text{ if } i &= j \text{ or } i = j. \\
d_i s_j &= s_j d_{i-1}   &\text{ if } i &> j. \\
s_i s_j &= s_{j+1} s_i   &\text{ if } i &≤ j.
\end{align*}

We call an element $\sigma \in X_n$ a **simplex of dimension $n$** or simply an **$n$-simplex**. We say that $\sigma$ is a **face** of a simplex $\tau$ if it is the image of $\tau$ via a composition of face maps. We say that $\sigma$ is **degenerate** if it is the image via a degeneracy map of any simplex. 

A simplicial set is said to be **computable** if for any tuple of $0$-simplices there exists at most one simplex having it as its ordered collection of $0$-dimensional faces. Examples include: representable simplicial sets, ordered or directed simplicial complexes, and the nerve of categories.

## Simplicial operators

The group of **simplicial operators** is the free [group generated](https://en.wikipedia.org/wiki/Presentation_of_a_group) by all symbols $s_i$ and $d_j$ modulo the simplicial identities. We refer to the product in this group as **composition** of simplicial operators.

The **canonical representative** of a simplicial operators has the form

\begin{equation*}
s_{u_1} \cdots s_{u_p} d_{v_1} \cdots d_{v_q}
\end{equation*}

with $u_1 > \cdots > u_p$ and $v_1 < \cdots < v_q$.

If $[v_0, \dots, v_n]$ represents an $n$-simplex in a computable simplicial set. The action of any simplicial operator on it is defined by

\begin{equation}
d_i [v_0, \dots, v_n] = [v_0, \dots, \widehat{v}_i, \dots, v_n]
\end{equation}

\begin{equation}
s_i [v_0, \dots, v_n] = [v_0, \dots, v_i, v_i, \dots, v_n].
\end{equation}

We model simplicial operators with the class `Operator`.

In [ ]:
from simplicial_operators import Operator

### Example

We will model the
$$
s_0s_1d_1d_0.
$$
Its canonical representative is 
$$
s_2s_0d_0d_2
$$

Its action on the simplex $[0,1,2,3]$ yields $[1, 1, 3, 3]$.

Its composition with itself is itself. I.e., it is idempotent.

In [ ]:
op = Operator([0,1],[1,0])

print(f'The canonical representative {op} of s_0s_1d_1d_0 is stored.\n')

print(f'Its action on [0, 1, 2, 3] is {list(op(range(4)))}\n')

print(f'We verify this operator is idempotent. Its composition with itself is {op.compose(op)}.')

## Chain complexes, tensor products and linear multioperators

Throughout this notebook all algebraic constructions are taken over the field $\mathbb F_2$ with two elements. 

A chain complex is a pair $(C_*, \partial)$ where 
$$
C_* = \bigoplus_{n \in \mathbb Z} C_n \qquad 
\partial = \bigoplus_{n \in \mathbb Z} \partial_n
$$

and $\partial_n : C_{n+1} \to C_n$ is a $\mathbb F_2$-linear map such that $\partial_{n+1} \partial_n = 0$.

The **tensor product** of two chain complexes $C$ and $C'$ is defined by

$$
(C \otimes C')_n = \bigoplus_{i+j=n} C_i \otimes C'_j\ \qquad
\partial = \partial \otimes \mathrm{id} + \mathrm{id} \otimes \partial
$$

The **normalized chains** $N_*(X)$ associated to a simplicial set $X$ is defined by

\begin{equation}
N_n(X) = \frac{\mathbb F_2 \{ X_n \}}{\mathbb F_2 \{ s(X_{n-1}) \}} \ \qquad
\partial_n = \sum_{i=0}^{n} d_{i}
\end{equation}
where $s(X_{n-1}) = \bigcup_{i=0}^{n-1} s_i(X_{n-1})$.

A **simplicial multioperator** is a formal $\mathbb F_2$-linear combination of the form 

$$
\sum_i op_1^{i} \otimes \cdots \otimes op_n^{i}
$$

whose action on $N_*(X_1) \otimes \cdots \otimes N_*(X_n)$ is defined coordinatewise.

We model them via `set` of `tuples` of `Operator`. Addition over this field is modeled by the function `__xor__` whose syntax is `^`.

In [ ]:
# We introduce a tool to simplify the representation of linear multioperators

def display(multiop):
    '''tool to aid visualization of linear multioperators'''
    if isinstance(multiop, Operator):
        return str(multiop)
    if isinstance(multiop, tuple):
        return tuple(display(op) for op in multiop)
    if isinstance(multiop, set):
        return set(display(mop) for mop in multiop)
    else:
        raise TypeError('expected either: Operator, tuple of Operators, or set of these tuples')

### Example

We will model  
$$
s_0s_1d_1d_0 \otimes s_1 d_0 d_1\ +\ s_1d_2d_0 \otimes s_2 d_1.
$$

Its canonical representative is 
$$
s_1d_0d_3 \otimes s_2d_1 \ +\ s_2s_0d_0d_2 \otimes s_1d_0d_1.
$$

Its action on $[0,1,2,3] \otimes [0,1,2,3,4]$ yields
$$
[1, 2, 2] \otimes [0, 2, 3, 3, 4] \ +\ [1, 1, 3, 3] \otimes [2, 3, 3, 4].
$$

Its composition with itself is equal to

$$
s_1s_0d_0d_1d_3d_4 \otimes s_2s_1d_1d_2 \ +\ 
s_2s_0d_0d_1d_3 \otimes s_1s_0d_0d_1d_2 \ +\
s_2s_0d_0d_2 \otimes s_1d_0d_1d_2 \ +\ 
s_1d_0d_2 \otimes s_2d_0d_1
$$


In [ ]:
biops = {(Operator([0,1],[1,0]), Operator([1],[0,1])), (Operator([1],[2,0]), Operator([2],[1]))}

print(f'The canonical representative {display(biops)} is stored.\n')

action = set()
for biop in biops:
    action ^= {(biop[0](range(4)), biop[1](range(5)))}

print(f'Its action on [0,1,2,3] x [0,1,2,3,4] is {action} \n')

composition = set()
for left in biops:
    for right in biops:
        #print(display(left), display(right))
        composition ^= {( left[0].compose(right[0]), left[1].compose(right[1]) )}
    
print(f'The composition with itself is: \n{display(composition)}.')